# Polynomial Rings

* Author: Luca Remke
* Version: OSCAR version 0.14.0
* Last modified: January 26, 2024

This tutorial provides an introduction to commutative algebra in OSCAR.
The aim is to describe the functionalities for multivariate rings and ideals in these rings and to explain them with minimal examples.
In order to facilitate the introduction, some of the methods are not presented in their entirety.
For a full description of the topic, see https://docs.oscar-system.org/stable/CommutativeAlgebra/intro/.

The reader needs no prior knowledge of *OSCAR*.

Content:
1. [Creating Multivariate Polynomial Rings](#1.-Creating-Multivariate-Polynomial-Rings)
   - [1.A Constructors](#1.A-Constructors)
   - [1.B Interlude: Monomial Ordering](#1.B-Interlude:-Monomial-Ordering)
   - [1.C Coefficient Rings](#1.C-Coefficient-Rings)
2. [Graded Polynomial Rings](#2.-Graded-Polynomial-Rings)
   - [2.A Constructors](#2.A-Constructors)
   - [2.B Properties](#2.B-Properties)
3. [Data Associated to Multivariate Rings](#3.-Data-Associated-to-Multivariate-Rings)
4. [Multivariate Polynomials](#4.-Multivariate-Polynomials)
   - [4.A Constructors](#4.A-Constructors)
   - [4.B Special Polynomials](#4.B-Special-Polynomials)
   - [4.C Data Associated to Elements of Multivariate Rings](#4.C-Data-Associated-to-Elements-of-Multivariate-Rings)
5. [Homomorphisms of Multivariate Polynomial Rings](#5.-Homomorphisms-of-Multivariate-Polynomial-Rings)
6. [Ideals in Multivariate Polynomial Rings](#6.-Ideals-in-Multivariate-Polynomial-Rings)
   - [6.A Constructors](#6.A-Constructors)
   - [6.B Data associated to Ideals](#6.B-Data-associated-to-Ideals)
   - [6.C Operations on Ideals](#6.C-Operations-on-Ideals)
   - [6.D Properties of Ideals](#6.D-Properties-of-Ideals)
   - [6.E Decomposition of Ideals](#6.E-Decomposition-of-Ideals)
7. [Homogenization and Dehomogenization](#7.-Homogenization-and-Dehomogenization)
   - [7.A Homogenization](#7.A-Homogenization)
   - [7.B Dehomogenization](#7.B-Dehomogenization)
   - [7.C Examples](#7.C-Examples)
8. [Generating Special Ideals](#8.-Generating-Special-Ideals)

In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.14.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2024 by The OSCAR Development Team


# 1. Creating Multivariate Polynomial Rings

## 1.A Constructors

Multivariate polynomial rings are constructed using:

| Method |
| :----------- |
| `polynomial_ring(C::Ring, var_names::Vector{String}; ordering=:lex, cached = true)` |

Its return value is a tuple `R, vars` consisting of a polynomial ring `R` with coefficient ring `C` and a vector `vars` of generators corresponding to the strings in vector `var_names`. 

The parameter `ordering` defines the monomial order and is assigned the default value `lex`. 
Caching is used to ensure that a constructed ring is unique in the system.

Here are a few examples:

In [2]:
# polynomial ring Z[x,y,z]
Z_xyz, (x, y, z) = polynomial_ring(ZZ, ["x", "y", "z"])

(Multivariate polynomial ring in 3 variables over ZZ, ZZMPolyRingElem[x, y, z])

In [3]:
# polynomial ring Q[x]
Q_x, x = polynomial_ring(QQ, ["x"])

(Multivariate polynomial ring in 1 variable over QQ, QQMPolyRingElem[x])

In [4]:
# polynomial ring whose variables have multi-indices
R, (x, y, z) = polynomial_ring(QQ, "x" => (1:2, 1:3), "y" => 1:2, "z" => (1:1, 1:1, 1:1))

(Multivariate polynomial ring in 9 variables over QQ, QQMPolyRingElem[x[1, 1] x[1, 2] x[1, 3]; x[2, 1] x[2, 2] x[2, 3]], QQMPolyRingElem[y[1], y[2]], [z[1, 1, 1];;;])

## 1.B Interlude: Monomial Ordering

Each multivariate polynomial ring in `OSCAR` comes equipped with a monomial ordering according to which the polynomials are stored and displayed. Independently of this ordering, Gröbner bases and, more generally, standard bases can be computed with respect to any monomial ordering: The `groebner_basis` and `standard_basis` functions provided by `OSCAR` allow one to specify the desired monomial `ordering` as a keyword argument. Typically, however, the user does not have to worry about Gröbner (standard) bases: The functions discussed in this chapter compute such bases behind the scenes when needed. Once computed, each such basis is cached for later reuse.

Multivariate polynomial rings can also be graded. Most of the functions that follow apply to both graded and non-graded cases. We will usually only consider the non-graded case when giving minimal examples.


## 1.C Coefficient Rings

In OSCAR, Gröbner bases are implemented for multivariate polynomial rings over the fields and rings below:

- Ring of integers $\mathbb{Z}$ via `ZZ`.
- Field of rational numbers $\mathbb{Q}$ via `QQ`
- Finite fields $\mathbb{F}_p$ with prime $p$ via `GF(p)`
- Finite fields $\mathbb{F}_{p^n}$ with $p^n$ elements and prime $p$
- Simple algebraic extensions of $\mathbb{Q}$ or $\mathbb{F}_p$.
- Purely transcendental extensions of $\mathbb{Q}$ or $\mathbb{F}_p$.

For the construction of finite fields and algebraic or transcendental extensions the following methods are used:

| Method |
| :----------- |
| `FlintFiniteField(char::ZZRingElem, deg::Int, s::AbstractString; cached = true)` |
| `NumberField(f::fmpq_poly, s::Union{AbstractString, Char, Symbol}; cached::Bool = true, check::Bool = true)` |
| `NumberField(f::Poly{NumFieldElem}, s::String; cached::Bool = false, check::Bool = false) -> NumField, NumFieldElem` |
| `FractionField(R::Ring; cached=true)` |

- `FlintFiniteField`: Returns a tuple $(S,x)$ consisting of a finite field $S$ with characteristic `char` and positive degree `d` and a generator $x$. The characteristic must be a prime. If a Conway polynomial is known, the field is generated with this polynomial, otherwise a random sparse irreducible polynomial is used. The parameter `s` determines how the name of the generator $x$ is printed. Instead of `FlintFiniteField` the shorter `FiniteField` can be used.
<br></br>
- `NumberField`: Returns a tuple $(R, x)$ consisting of the algebraic number field $\mathbb{Q}[x]/(f)$, where `f` is the specified polynomial, and the generator $x$. The string `s` specifies how the name of the generator $x$ is printed.
<br></br>
- `NumberField`: Returns a tuple $(L, b)$ consisting of the simple algebraic number field $L = K[x]/(f)$ and the class `b` of $x$ in $L$. Here `f` is an irreducible polynomial in $K[x]$ over a number field $K$. The string `s` specifies how the primitive element $b$ is printed.
<br></br>
- `FractionField`: Returns the field of fractions of the ring `R`.

Let us exemplify this functionality:

In [5]:
ZZ

Integer ring

In [6]:
QQ

Rational field

In [7]:
GF(3)

Finite field of degree 1 over GF(3)

In [8]:
FiniteField(2, 70, "a")

(Finite field of degree 70 over GF(2), a)

In [9]:
T, t = polynomial_ring(GF(3), "t")
K, a = FiniteField(t^2 + 1, "a")

(Finite field of degree 2 over GF(3), a)

In [10]:
T, t = polynomial_ring(QQ, "t")
K, a = NumberField(t^2 + 1, "a")

(Number field of degree 2 over QQ, a)

In [11]:
T, t = polynomial_ring(QQ, "t")
QT = FractionField(T)

Fraction field
  of univariate polynomial ring in t over QQ

# 2. Graded Polynomial Rings

In the following, a ring $R$ which is graded by a finitely presented group $G$ is also called *$G$-graded* or *graded* and the group $G$ is called *graded group*. If $R$ is a polynomial ring over a field, a $G$-graded on $R$ is called *positive* if $G$ is free and each graded part $R_g, g \in G$ is of finite dimension. In this case $R$ is *positively graded*. Thus, with respect to OSCAR, we call a `G`-graded ring `R` *$\mathbb{Z}^m$-graded* if `is_free(G) && ngens(G) == rank(G) == m` evaluates to `true`. In this case we can identify elements of `G` and integer vectors of length `m`.

## 2.A Constructors

There are two ways to create multivariate rings with grading:

1. With `grade` a graduation is assigned to an already defined polynomial ring:

| Method |
| :----------- |
| `grade(R::MPolyRing, W::Vector{GrpAbFinGenElem})` |
| `grade(R::MPolyRing, W::Vector{<:Vector{<:IntegerUnion}})` |
| `grade(R::MPolyRing, W::Union{ZZMatrix, Matrix{<:IntegerUnion}})` |
| `grade(R::MPolyRing, W::Vector{<:IntegerUnion})` |
| `grade(R::MPolyRing)` |

In all cases, a graded ring is created together with the vector of its variables. 
For this purpose, a `G` grading on the ring `R` is defined from the vector `W` with `ngens(R)` elements of a group `G` by assigning weights to the variables corresponding to the entries of `W`. 
If integer vectors of the same length `m` are specified instead of group elements, a $\mathbb{Z}^m$-grading is defined on the ring `R`. 
Without specifying `W`, `R` is assigned the standard $\mathbb{Z}$-grading.

2. With `graded_polynomial_ring` a polynomial ring is created together with a grading:

| Method |
| :----------- |
| `graded_polynomial_ring(C::Ring, V::Vector{String}, W; ordering=:lex)` |
| `graded_polynomial_ring(C::Ring, V::Vector{String}; ordering=:lex)` |

The methods create a graded multivariate polynomial ring with coefficient ring `C` together with the vector of its variables whose names are determined by the strings in `V`. 
The grading of this ring is defined with the data provided by `W`, see `grade`. 
Without specifying `W`, `R` receives the standard $\mathbb{Z}$-grading.

Time for examples:

In [12]:
# polynomial ring Q[t,x,y]
R, (t, x, y) = polynomial_ring(QQ, ["t", "x", "y"])

# assigning a grading
S, (t, x, y) = grade(R, [-1, 1, 1])

(Graded multivariate polynomial ring in 3 variables over QQ, MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}[t, x, y])

In [13]:
# a graded polynomial ring
T, (t, x, y) = graded_polynomial_ring(QQ, ["t", "x", "y"], [-1, 1, 1])

(Graded multivariate polynomial ring in 3 variables over QQ, MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}[t, x, y])

## 2.B Properties

| Method | Description |
| :----------- | :----------- |
| `is_standard_graded(R::MPolyRing_dec)` | Checks whether `R` is standard-$\mathbb{Z}$-graded. |
| `is_z_graded(R::MPolyRing_dec)` | Checks whether `R` is $\mathbb{Z}$-graded. |
| `is_zm_graded(R::MPolyRing_dec)` | Checks whether `R` is $\mathbb{Z}^m$-graded. |
| `is_positively_graded(R::MPolyRing_dec)` | Checks whether `R` is positively graded. |

##### Example

In [14]:
T, (t, x, y) = graded_polynomial_ring(QQ, ["t", "x", "y"], [-1, 1, 1])
(is_standard_graded(T), is_z_graded(T), is_positively_graded(T))

(false, true, false)

# 3. Data Associated to Multivariate Rings

The following information can be retrieved for a multivariate polynomial ring `R` with coefficient ring `C`:

| Method | Description |
| :----------- | :----------- |
| `coefficient_ring(R)` | Coefficient ring `C`. |
| `gens(R)` | Generators (variables) of `R`. |
| `ngens(R)` | Number of the generators of `R`. |
| `gen(R, i)` | `i`-th generator of `R`. |

In the graded case, we additionally have:

| Method | Description |
| :----------- | :----------- |
| `grading_group(R::MPolyDecRing)` | `G`-grading of `R` |
| `homogeneous_component(R::MPolyDecRing,g::GrpAbFinGenElem)` <br><br> `homogeneous_component(R::MPolyDecRing,g::Vector{<:IntegerUnion})` <br><br> `homogeneous_component(R::MPolyDecRing,g::IntegerUnion)` | Homogeneous component of the polynomial ring `R` of degree `g` together with the embedding of the components into `R`. The type of `g` depends on the grading of `R`. |

Here are a few examples:

In [15]:
R, (x, y, z) = polynomial_ring(QQ, ["x", "y", "z"])
(coefficient_ring(R), gens(R), ngens(R), gen(R, 1), gen(R, 2), gen(R, 3))

(Rational field, QQMPolyRingElem[x, y, z], 3, x, y, z)

In [16]:
R, (x, y, z) = graded_polynomial_ring(QQ, ["x", "y", "z"], [1, 2, 3])
grading_group(R)

GrpAb: Z

In [17]:
R, (x, y, z) = graded_polynomial_ring(QQ, ["x", "y", "z"], [1, 2, 3])
homogeneous_component(R, 1)

(homogeneous component of graded multivariate polynomial ring in 3 variables over QQ of degree [1], Map: r_[1] of dim 1 -> graded multivariate polynomial ring)

# 4. Multivariate Polynomials

## 4.A Constructors

There are the following possibilities for constructing elements of a polynomial ring:
- Build up polynomials from the generators of the ring using basic arithmetic.
- Use `MPolyBuildCtx(R::MPolyRing)`, returning a *build context*.
- Use

    | Method |
    | :----------- |
    | `(R::MPolyRing{T})(c::Vector{T}, e::Vector{Vector{Int}}) where T <: RingElem` |
    
    This generates an element of `R` with non-trivial coefficients given by the vector `c` and exponent vectors given by the elements of `e`.

Time for examples:

In [18]:
R, (x, y, z) = polynomial_ring(QQ, ["x", "y", "z"])

# using basic arithmetic
f = 3*x^2+y*z

# Using the method
g = R(QQ.([3, 1]), [[2, 0, 0], [0, 1, 1]])

# Using MPolyBuildCtx
B = MPolyBuildCtx(R)
push_term!(B, 3, [2, 0, 0])
push_term!(B, 1, [0, 1, 1])
h = finish(B)

f == g == h

true

## 4.B Special Polynomials

For a multivariate polynomial ring `R`, `zero(R)` and `one(R)` denote the additive and multiplicative neutral elements of `R`, respectively. 
With `is_zero(f)` or `is_one(f)` these can be checked for an element `f`.

## 4.C Data Associated to Elements of Multivariate Rings

The following information can be obtained from an element `f` of a multivariate ring `R`:

| Method | Description |
| :----------- | :----------- |
| `parent(f)` | Ring `R`. |
| `total_degree(f)` | Total degree of `f`. |
| `monomial(f, i)` | `i`-th monomial of `f`. |
| `term(f, i)` | `i`-th term of `f`. |
| `coeff(f, i)` | `i`-th coefficient of `f`. |
| `exponent_vector(f, i)` | Exponent vector of the `i`-th term of `f`. |

Further functionality is available in the graded case:

| Method | Description |
| :----------- | :----------- |
| `homogeneous_components(f::MPolyDecRingElem{T, S}) where {T, S}` <br><br> `homogeneous_component(f::MPolyDecRingElem,g::GrpAbFinGenElem)` <br><br> `homogeneous_component(f::MPolyDecRingElem,g::Vector{<:IntegerUnion})` <br><br> `homogeneous_component(f::MPolyDecRingElem, g::IntegerUnion)` | Homogeneous components of `f` of degree `g`. The type of `g` depends on the type of grading of `R`. |
| `is_homogeneous(f::MPolyDecRingElem)` | Checks whether `f` is homogeneous. |
| `degree(f::MPolyDecRingElem)` | Degree of `f`. |
| `degree(::Type{Vector{Int}}, f::MPolyDecRingElem)` | Degree of a homogeneous element `f` of a $\mathbb{Z}^m$-graded multivariate ring, converted to a vector of integer numbers. |
| `degree(::Type{Int}, f::MPolyDecRingElem)` | Degree of a homogeneous element `f` of a $\mathbb{Z}$-graded multivariate ring, converted to an integer number. |

##### Example

In [19]:
R, (x, y, z) = polynomial_ring(QQ, ["x", "y", "z"])
f = 3*x^2 + y*z
(total_degree(f), coeff(f, 2), exponent_vector(f, 2), monomial(f, 2), term(f, 2))

(2, 1, [0, 1, 1], y*z, y*z)

In [20]:
R, (x, y, z) = graded_polynomial_ring(QQ, ["x", "y", "z"], [1, 2, 3])
f = 3*x^2 + y*z
homogeneous_components(f)

Dict{GrpAbFinGenElem, MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}} with 2 entries:
  [2] => 3*x^2
  [5] => y*z

# 5. Homomorphisms of Multivariate Polynomial Rings

If $R$ is a multivariate polynomial ring, and $S$ is any ring, then a ring homomorphism $R \rightarrow S$ is determined by specifying its restriction to the coefficient ring of $R$, and by assigning an image to each variable of $R$. 
In OSCAR, such homomorphisms are created by using the following constructor:

| Method |
| :----------- |
| `hom(R::MPolyRing, S::NCRing, coeff_map, images::Vector; check::Bool = true)` |
| `hom(R::MPolyRing, S::NCRing, images::Vector; check::Bool = true)` |


Given a homomorphism `coeff_map` from `C` to `S`, where `C` is the coefficient ring of `R`, and given a vector `images` of `nvars(R)` elements of `S`, return the homomorphism from `R` to `S` whose restriction to `C` is `coeff_map`, and which sends the $i$-th variable of `R` to the $i$-th entry of `images`.

If no coefficient map is entered, invoke a canonical homomorphism of `C` to `S`, if such a homomorphism exists, and throw an error, otherwise.

In case `check = true` (default), the function checks the conditions below:
- If `S` is graded, the assigned images must be homogeneous with respect to the given grading.
- If `S` is noncommutative, the assigned images must pairwise commute. 

If $F: R \rightarrow S$ is a ring morphism, then `domain(F)`=`R` und `codomain(F)` = `S`.

##### Example

In [21]:
R, (x, y) = polynomial_ring(ZZ, ["x", "y"])
f = 3*x^2 + 2*x + 1

S, (x, y) = polynomial_ring(GF(2), ["x", "y"])

# morphism from R to S
H = hom(R, S, gens(S))
H(f)

x^2 + 1

# 6. Ideals in Multivariate Polynomial Rings

## 6.A Constructors

The construction of ideals in multivariate polynomial rings is done by the method:

| Method |
| :----------- |
| `ideal(R::MPolyRing, g::Vector)` |

where `g` is a vector of polynomials in the ring `R`. 
The return is the ideal generated by the polynomials. 
In the graded case these polynomials must be homogeneous.

Here is an example:

In [22]:
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
I = ideal(R, [x*y-3*x, y^3-2*x^2*y])

ideal(x*y - 3*x, -2*x^2*y + y^3)

## 6.B Data associated to Ideals

If `I` is an ideal of a multivariate polynomial ring `R`, then

| Method | Description |
| :----------- | :----------- |
| `base_ring(I)` | `R`. |
| `gens(I)` | Generators `I`. |
| `ngens(I)` | Number of generator of `I`. |
| `gen(I, k)` or `I[k]` | `k`-th generator of `I`. |
| `dim(I::MPolyIdeal)` | Krull dimension of `I`. |
| `codim(I::MPolyIdeal)` | Codimension of `I`. |
| `minimal_generating_set(I::MPolyIdeal{<:MPolyDecRingElem})` | Array containing a minimal set of generators of `I`. |

In [23]:
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
I = ideal(R, [x, y])^2
(base_ring(I), gens(I), ngens(I), gen(I, 2))

(Multivariate polynomial ring in 2 variables over QQ, QQMPolyRingElem[x^2, x*y, y^2], 3, x*y)

In [24]:
R, (x, y, z) = graded_polynomial_ring(QQ, ["x", "y", "z"]);
V = [x, z^2, x^3+y^3, y^4, y*z^5];
I = ideal(R, V)

(dim(I), codim(I), minimal_generating_set(I))

(0, 3, MPolyDecRingElem{QQFieldElem, QQMPolyRingElem}[x, z^2, y^3])

## 6.C Operations on Ideals

Sums, products and powers of ideals can be formed via `+`, `*` and `^` respectively. 
Furthermore, there are the following methods:

| Method | Description |
| :----------- | :----------- |
| `intersect(I::MPolyIdeal{T}, Js::MPolyIdeal{T}...)` | Intersection of two or more ideals. |
| `quotient(I::MPolyIdeal{T}, J::MPolyIdeal{T})` | Ideal quotient $I:J = \{ f \in R \vert fJ \subset I\} \subset R$ of `I` by `J`. Alternatively, use `I:J`. |
| `quotient(I::MPolyIdeal{T}, f::MPolyElem{T})` | Ideal quotient of `I` by the ideal generated by `f`. ALternatively, use `I:f`. |
| `saturation(I::MPolyIdeal{T}, J::MPolyIdeal{T})` | Saturation $I:J^{\infty} = \{f \in R \vert fJ^k \subset I , k \geq 1\}$ of `I` wih respect to `J`. |
| `saturation_with_index(I::MPolyIdeal{T},J::MPolyIdeal{T})` | Saturation $I:J^\infty$ together with the smallest integer $m$ such that $I:J^m=I:J^\infty$. |
| `eliminate(I::MPolyIdeal{T}, V::Vector{T})` | Eliminates the variables in the vector `V` from the ideal `I` and returns all generators of `I` that contain only the remaining variables. |

Let us demonstrate this functionality:

In [25]:
R, (x, y, z) = polynomial_ring(QQ, ["x", "y", "z"])
I = ideal(R, [x, y])
J = ideal(R, [z^2])

(I^3, I+J, I*J)

(ideal(x^3, x^2*y, x*y^2, y^3), ideal(x, y, z^2), ideal(x*z^2, y*z^2))

In [26]:
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
I = ideal(R, [y^2-x^3+x, x*y])
J = ideal(R, [x, y])^2

[intersect(I, J), quotient(I, J), saturation(I, J), eliminate(I, [y])]

4-element Vector{MPolyIdeal{QQMPolyRingElem}}:
 ideal(x*y, y^3, x^4 - x^2)
 ideal(y, x^3 - x)
 ideal(y, x^2 - 1)
 ideal(x^4 - x^2)

## 6.D Properties of Ideals

The following properties of an ideal can be tested:

| Method | Description |
| :----------- | :----------- |
| `is_zero(I::MPolyIdeal)` | Checks whether `I` is the zero ideal. |
| `is_one(I::MPolyIdeal)` | Checks whether `I` is generated by $1$. |
| `is_monomial(I::MPolyIdeal)` | Checks whether `I` is generated by monomials. |
| `is_subset(I::MPolyIdeal{T}, J::MPolyIdeal{T})` | Checks whether `I` is contained in `J`. |
| `==(I::MPolyIdeal{T}, J::MPolyIdeal{T})` | Checks whether `I` is equal to `J`. |
| `ideal_membership(f::T, I::MPolyIdeal{T})` | Checks whether `f` is contained in `I`. |
| `radical_membership(f::T, I::MPolyIdeal{T})` | Checks whether `f` is contained in the radical of `I`. |
| `is_prime(I::MPolyIdeal)` | Checks whether `I` is prime. |
| `is_primary(I::MPolyIdeal)` | Checks whether `I` is primary. |

##### Example

In [27]:
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
I = ideal(R, [x, y])^2
J = ideal(R, [x^2])
f = x

(is_monomial(I), is_subset(J, I), ideal_membership(f, J), radical_membership(f,J), is_prime(J), is_primary(J))

(true, true, false, true, false, true)

## 6.E Decomposition of Ideals

The following methods can be used to decompose an ideal in a polynomial ring over a field:

| Method | Description |
| :----------- | :----------- |
| `radical(I::MPolyIdeal)` | Radical of `I`. |
| `primary_decomposition(I::MPolyIdeal; alg = :GTZ)` | Minimal primary decomposition of `I`, given as vector of tuples $(Q_i, P_i)$ where $Q_i$ is a primary ideal with associated prime ideal $P_i$. |
| `absolute_primary_decomposition(I::MPolyIdeal{<:MPolyElem{fmpq}})` | Absolute minimal primary decomposition of `I`. |
| `minimal_primes(I::MPolyIdeal; alg = :GTZ)` | Vector containing the minimal associated prime ideals of `I`. Alternatively, use `alg = :charSets`. |
| `equidimensional_decomposition_weak(I::MPolyIdeal)` | Vector of equidimensional ideals where the last entry is the equidimensional hull of `I`, that is, the intersection of the primary components of `I` of maximal dimension. |
| `equidimensional_decomposition_radical(I::MPolyIdeal)` | Vector of equidimensional radical ideals increasingly ordered by dimension. For each dimension, the returned radical ideal is the intersection of the associated primes of `I` of that dimension. |
| `equidimensional_hull(I::MPolyIdeal)` | Intersection of the primary components of `I` of maximal dimension. In the case of polynomials over the integers, return the intersection of the primary components of `I` of minimal height.|
| `equidimensional_hull_radical(I::MPolyIdeal)` | Intersection of the associated primes of I of maximal dimension.  |

##### Example

In [28]:
# radical ideal in a polynomial ring
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
I = intersect(ideal(R, [x, y])^2, ideal(R, [y^2-x^3+x]), ideal(R, [x-y-1])^2)
radical(I)

ideal(x^4 - x^3*y - x^3 - x^2 - x*y^2 + x*y + x + y^3 + y^2)

In [29]:
# primary decomposition of an ideal
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
I = intersect(ideal(R, [x, y])^2, ideal(R, [y^2-x^3+x]), ideal(R, [x-y-1])^2)
primary_decomposition(I)

3-element Vector{Tuple{MPolyIdeal{QQMPolyRingElem}, MPolyIdeal{QQMPolyRingElem}}}:
 (ideal(x^3 - x - y^2), ideal(x^3 - x - y^2))
 (ideal(x^2 - 2*x*y - 2*x + y^2 + 2*y + 1), ideal(x - y - 1))
 (ideal(y, x^2), ideal(x, y))

In [30]:
# minimal associated primare ideals
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
I = intersect(ideal(R, [x, y])^2, ideal(R, [y^2-x^3+x]), ideal(R, [x-y-1])^2)
minimal_primes(I)

2-element Vector{MPolyIdeal{QQMPolyRingElem}}:
 ideal(x - y - 1)
 ideal(x^3 - x - y^2)

In [31]:
# equidimensional ideals
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
I = intersect(ideal(R, [x, y])^2, ideal(R, [y^2-x^3+x]), ideal(R, [x-y-1])^2)
equidimensional_decomposition_weak(I)

2-element Vector{MPolyIdeal{QQMPolyRingElem}}:
 ideal(y, x)
 ideal(x^5 - 2*x^4*y - 2*x^4 + x^3*y^2 + 2*x^3*y - x^2*y^2 + 2*x^2*y + 2*x^2 + 2*x*y^3 + x*y^2 - 2*x*y - x - y^4 - 2*y^3 - y^2)

In [32]:
# equidimensional radical ideals
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
I = intersect(ideal(R, [x, y])^2, ideal(R, [y^2-x^3+x]), ideal(R, [x-y-1])^2)
equidimensional_decomposition_radical(I)

2-element Vector{MPolyIdeal{QQMPolyRingElem}}:
 ideal(y, x)
 ideal(x^4 - x^3*y - x^3 - x^2 - x*y^2 + x*y + x + y^3 + y^2)

# 7. Homogenization and Dehomogenization

## 7.A Homogenization

The homogenization of an element of a $\mathbb{Z}^m$-graded polynomial ring is done by the following methods:

| Method |
| :----------- |
| `homogenization(f::MPolyRingElem, W::Union{ZZMatrix, Matrix{<:IntegerUnion}}, var::String, pos::Int = 1)` |
| `homogenization(V::Vector{T},  W::Union{ZZMatrix, Matrix{<:IntegerUnion}}, var::String, pos::Int = 1) where {T <: MPolyRingElem}` |
| `homogenization(I::MPolyIdeal{T},  W::Union{ZZMatrix, Matrix{<:IntegerUnion}}, var::String, pos::Int = 1) where {T <: MPolyRingElem}` |

If $m$ is the number of rows of `W`, extend the parent polynomial ring of `f` by inserting $m$ extra variables, starting at position `pos`. 
Correspondingly, extend the integer matrix `W` by inserting the standard unit vectors of size mm as new columns, starting at column `pos`. 
Grade the extended ring by converting the columns of the extended matrix to elements of the group $\mathbb{Z}^m$ and assigning these as weights to the variables. 
Homogenize `f` with respect to the induced ZmZm-grading on the original ring, using the extra variables as homogenizing variables. 
Return the result as an element of the extended ring with its $\mathbb{Z}^m$-grading. 
If $m=1$, the extra variable prints as `var`. 
Otherwise, the extra variables print as `var[i]`, for $i=1,...,m$.

Multiple elements can be homogenized in this way by specifying them as a vector `V`. 
For an ideal `I`, the generators of `I` are homogenized and then the ideal generated by these homogenizations is defined in the extended polynomial ring. 
This ideal is then saturated with the ideal generated by the additional homogenised variables.

## 7.B Dehomogenization

The dehomogenization of an element of a $\mathbb{Z}^m$-graded polynomial ring is done by the following methods:

| Method |
| :----------- |
| `dehomogenization(F::MPolyDecRingElem, pos::Int)` |
| `dehomogenization(V::Vector{T}, pos::Int) where {T <: MPolyDecRingElem}` |
| `dehomogenization(I::MPolyIdeal{T}, pos::Int) where {T <: MPolyDecRingElem}` |

Given an element `F` of a $\mathbb{Z}^m$-graded ring, where the generators of $\mathbb{Z}^m$ are the assigned weights to the variables at positions `pos`, ..., `pos`$-1+m$, dehomogenize `F` using the variables at these positions. 
Return the result as an element of a polynomial ring not depending on the variables at these positions.

Multiple elements can be dehomogenized by specifying them as a vector `V`. 
Similarly, an ideal `I` can be dehomogenized.

## 7.C Examples

In [33]:
# homogenization
R, (x, y) = polynomial_ring(QQ, ["x", "y"])
f = x^3+x^2*y+x*y^2+y^3
W = [1 2; 3 4]
L = homogenization(f, W, "z", pos = 3)
[L, parent(L)]

2-element Vector{Any}:
 x^3*z_{1}^3*z_{2}^3 + x^2*y*z_{1}^2*z_{2}^2 + x*y^2*z_{1}*z_{2} + y^3
 Graded multivariate polynomial ring in 4 variables over QQ

In [34]:
# dehomogenization
S, (x, y, z) = graded_polynomial_ring(QQ, ["x", "y", "z"])
F = x^3-x^2*y-x*z^2
f = dehomogenization(F, 1)
[f, parent(f)]

2-element Vector{Any}:
 -y - z^2 + 1
 Multivariate polynomial ring in 2 variables over QQ

# 8. Generating Special Ideals

- The katsura ideal `katsura(n)` is generated by $u_m - \sum_{l=n}^{m} u_{l-m}u_l$ and $1-\sum_{l=-n}^{n} u_l$ where `n` is a natural number, $u_{-i}=u_i$, $u_i=0$ for all $i>n$ and $m \in \{-n,...,n\}$.

| Method | Description |
| :----------- | :----------- |
| `katsura(n::Int)` | Katsura ideal. |
| `katsura(R::MPolyRing)` | Katsura ideal in the given polynomial ring `R`. |

Here is a simple example:

In [35]:
katsura(2)

ideal(x1 + 2*x2 + 2*x3 - 1, x1^2 - x1 + 2*x2^2 + 2*x3^2, 2*x1*x2 + 2*x2*x3 - x2)